
    
# Converting a TensorFlow* Model in Linux*

## Pre-Requisites:


    
### Install Tensorflow 1.12 version

Run the following cells to install tensorflow 1.12 version

In [23]:
pip install --upgrade pip

Requirement already up-to-date: pip in /home/u26212/.local/lib/python3.5/site-packages (19.2.3)
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.

In [24]:
pip install --user tensorflow==1.12

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.

Create new temporary directory by running the following cell

In [25]:
! export TMPDIR=/tmp/$USER; mkdir -p $TMPDIR; python -c "import tempfile; print(tempfile.gettempdir())"

/tmp/u26212


In [26]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1 -o models


###############|| Downloading topologies ||###############

========= Downloading models/classification/squeezenet/1.1/caffe/squeezenet1.1.prototxt
... 100%, 9 KB, 37936 KB/s, 0 seconds passed

========= Downloading models/classification/squeezenet/1.1/caffe/squeezenet1.1.caffemodel
... 100%, 4834 KB, 27145 KB/s, 0 seconds passed


###############|| Post processing ||###############

========= Changing input dimensions in squeezenet1.1.prototxt =========


### Download Models from TensorFlow* -slim library

There are a number of pre-trained public models in the TensorFlow*-slim repository. The models are distributed as Python scripts and checkpoint files. First of all download repository with models. Note: This is done in home directory(~)

In [27]:
! cd && git clone https://github.com/tensorflow/models/ && cd models/research/slim

fatal: destination path 'models' already exists and is not an empty directory.



    
### Export Inference Graph and download checkpoint file

Export inference graph for one of the available models using the following command (Inception V1 in this example):

In [28]:
! python3 -W ignore  $HOME/models/research/slim/export_inference_graph.py --alsologtostderr --model_name=inception_v1 --output_file=/tmp/$USER/inception_v1_inf_graph.pb  

INFO:tensorflow:Scale of 0 disables regularizer.



    
The script creates inference graph file with name “inception_v1_inf_graph.pb” in the /tmp direcory.

### Download archive with checkpoint file (Inception V1 in this example):

In [7]:
%%writefile run.sh
export CHECKPOINT_DIR=/tmp/$USER/checkpoints

mkdir $CHECKPOINT_DIR

wget http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz

tar -xvf inception_v1_2016_08_28.tar.gz

mv inception_v1.ckpt $CHECKPOINT_DIR

rm inception_v1_2016_08_28.tar.gz

Writing run.sh


In [29]:
%%bash
sh run.sh

inception_v1.ckpt


mkdir: cannot create directory ‘/tmp/u26212/checkpoints’: File exists
--2019-08-30 07:50:36--  http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.193.144, 2607:f8b0:4000:808::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.193.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24642554 (24M) [application/x-tar]
Saving to: ‘inception_v1_2016_08_28.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  573K 42s
    50K .......... .......... .......... .......... ..........  0% 1.17M 31s
   100K .......... .......... .......... .......... ..........  0% 2.06M 24s
   150K .......... .......... .......... .......... ..........  0% 2.48M 21s
   200K .......... .......... .......... .......... ..........  1% 3.76M 18s
   250K .......... .......... .......... .......... ..........  1% 4.74M 15s
   300K .......... .......... ........


    
### Freeze the model

The last step is to freeze the graph. To do this you need to know the output node of the model you are planning to freeze. This information is found by running the summarize_graph.

### Summarize Graph

Go to ~/models/research/slim/ directory and run summarize_graph.py script.

In [30]:
! cd $HOME/models/research/slim/ && python3 -W ignore  /opt/intel/openvino/deployment_tools/model_optimizer/mo/utils/summarize_graph.py --input_model=/tmp/$USER/inception_v1_inf_graph.pb 


1 input(s) detected:
Name: input, type: float32, shape: (-1,224,224,3)
1 output(s) detected:
InceptionV1/Logits/Predictions/Reshape_1



    
The output layer/node name should be in the last line of text on the console and should look like:

1 input(s) detected: Name: input, type: float32, shape: (-1,224,224,3) 1 output(s) detected: InceptionV1/Logits/Predictions/Reshape_1

### Freeze the graph for Intel® Distribution of OpenVINO™ toolkit

The script generates inception_v1_frozen.pb file with the frozen model in the directory you are currently in.

In [31]:
! python3 -W ignore /usr/local/lib/python3.5/dist-packages/tensorflow/python/tools/freeze_graph.py --input_graph /tmp/$USER/inception_v1_inf_graph.pb --input_binary --input_checkpoint /tmp/$USER/checkpoints/inception_v1.ckpt --output_node_names InceptionV1/Logits/Predictions/Reshape_1 --output_graph inception_v1_frozen.pb

Instructions for updating:
Use tf.gfile.GFile.
2019-08-30 07:51:20.990133: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA



    
You might get warning message "Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA" while executing above command, ignore that.
## Convert Frozen Tensorflow* model to IR using Model Optimizer

Assuming you are in the ~/models/research/slim/ directory

In [32]:
! python3 -W ignore  /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model inception_v1_frozen.pb --input_shape [1,224,224,3] --mean_values [1024,1024,1024] --scale_values [128,128,128]

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u26212/30/iot-devcloud/smart_video_workshop/object-detection/Python/inception_v1_frozen.pb
	- Path for generated IR: 	/home/u26212/30/iot-devcloud/smart_video_workshop/object-detection/Python/.
	- IR output name: 	inception_v1_frozen
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,224,224,3]
	- Mean values: 	[1024,1024,1024]
	- Scale values: 	[128,128,128]
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers imple


    
This should produce “inception_v1_frozen.xml” and “inception_v1_frozen.bin” file. The xml file contains the topology information of the model and the bin file contains the model’s weights and biases. These two files are expected when using the inference engine and so make note of the path.

## Run Classification Sample

The classification sample will showcase the Intel® Distribution of OpenVINO™ toolkit inference engine using TensorFlow model Inception_v1_frozen IR files (.xml & .bin) and an input image of a car to classify. The classification collateral is defined as the input image car_1.bmp, the Inception_v1_frozen IR files (.xml & .bin), and the labels file inception_v1_frozen.labels. Create a new directory that will hold the classification sample app and all needed components to run the classification sample Note: The following steps should be followed and are assuming you are following the preceding steps. You should be in the home directory.

In [33]:
!/opt/intel/openvino/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


In [34]:
! cp /opt/intel/openvino/deployment_tools/demo/car_1.bmp  . >/dev/null 2>&1

In [35]:
! cp /opt/intel/openvino/deployment_tools/demo/squeezenet1.1.labels ./inception_v1_frozen.labels >/dev/null 2>&1


    
#### Run Application

Note: To see all the flags that the sample takes as input, run the below cell

In [36]:
! python3 classification_sample.py -h

usage: classification_sample.py [-h] -m MODEL -i INPUT [INPUT ...]
                                [-l CPU_EXTENSION] [-pp PLUGIN_DIR]
                                [-d DEVICE] [--labels LABELS] [-nt NUMBER_TOP]
                                [-ni NUMBER_ITER] [-pc]

optional arguments:
  -h, --help            show this help message and exit
  -m MODEL, --model MODEL
                        Path to an .xml file with a trained model.
  -i INPUT [INPUT ...], --input INPUT [INPUT ...]
                        Path to a folder with images or path to an image files
  -l CPU_EXTENSION, --cpu_extension CPU_EXTENSION
                        MKLDNN (CPU)-targeted custom layers.Absolute path to a
                        shared library with the kernels impl.
  -pp PLUGIN_DIR, --plugin_dir PLUGIN_DIR
                        Path to a plugin folder
  -d DEVICE, --device DEVICE
                        Specify the target device to infer on; CPU, GPU, FPGA
                        or MYRIAD is accept

In [38]:
! python3 classification_sample.py -i car_1.bmp --labels inception_v1_frozen.labels -m inception_v1_frozen.xml

[ INFO ] Loading network files:
	inception_v1_frozen.xml
	inception_v1_frozen.bin
[ INFO ] Preparing input blobs
size is 1
[ WARNING ] Image car_1.bmp is resized from (637, 749) to (224, 224)
[ INFO ] Batch size is 1
[ INFO ] Loading model to the plugin
[ INFO ] Starting inference (1 iterations)
[ INFO ] Processing output blob
[ INFO ] Top 10 results: 
Image car_1.bmp

829  0.6013383 label tram, tramcar, trolley, trolley car
905  0.1193139 label shade
557  0.0587129 label flagstaff
600  0.0273736 label claw
812  0.0096623 label shuttle
633  0.0080024 label jeweler's loupe
795  0.0052693 label ski
791  0.0045856 label cart
652  0.0042737 label uniform
689  0.0042264 label overskirt


[ INFO ] Average running time of one iteration: 48.58970642089844 ms
[ INFO ] total running time of inference: 48.58970642089844 ms
[ INFO ] Throughput: 20.580490677134446 FPS


[ INFO ] Execution successful




Expected Output:


 
<img src="classification_output.png"> 
